# Building a basic SPAM Filter with Naive Bayes 

**dataset source:** University of California Irvine Spam DATABASE 'UCIspamdb.csv'

The collection is composed by just one text file, where each line has the correct class followed by the raw message. Some examples below:

ham or spam?  | email/sms text content
--------------|------------------
ham       | What you doing?how are you?
ham       |  Cos i was out shopping wif darren jus now n i called him 2 ask wat present he wan lor. Then he started guessing who i was wif n he finally guessed darren lor.
spam | Sunshine Quiz! Win a super Sony DVD recorder if you canname the capital of Australia? Text MQUIZ to 82277. B
spam | FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! ubscribe6GBP/ mnth inc 3hrs 16 stop?txtStop

In [1]:
# Import our usual packages...
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

# Import some useful packages from scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB # The Naive Bayes algo

In [2]:
# connect to Google Drive and load the data
# from google.colab import drive
# drive.mount('/content/drive')
df = pd.read_csv('../data/UCIspamdb.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5514 entries, 0 to 5513
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  5514 non-null   object
 1   text    5514 non-null   object
dtypes: object(2)
memory usage: 86.3+ KB


In [3]:
# Display the first 10 rows of the dataframe
df.head(10)

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [4]:
# replace the "spam" or "ham" tag with 1,0, using the numpy where() function
df['target'] = np.where(df['target']=='spam',1, 0)

In [5]:
# Display the first 10 rows of the dataframe
df.head(10)

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
5,1,FreeMsg Hey there darling it's been 3 week's n...
6,0,Even my brother is not like to speak with me. ...
7,0,As per your request 'Melle Melle (Oru Minnamin...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...


In [6]:
# Define inputs and target
X=df['text']
y=df['target']

In [7]:
# split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.3, random_state=0)

## Vectorize Words

Because we are dealing with text, we need to an intertmediate step to **vectorize** the content, meaning turn it into digits. For example, imagine we have in our database entries:

$x_1$: `"Prescription drugs! Buy cheap prescription drugs drugs with big discount!"`

$x_2$: `"Your prescription drugs from CVS are ready for pickup."`

Step 1. We start by building a giant dictionary of words. Each row of text is converted into a vector of numbers, as follows:

text | discount | ... | big | ...| buy | ... | cheap | ... | drugs | ... | pickup | ... | prescription | ... | ;is spam?
-----|----|-----|-----|-----|---|-----|-------|-----|-------|-----|---|---|---|---|---
$x_1$ | 1 | ... | 1 | ... | 1 | ... | 1 | ... | 3 | ... | 0 | ... | 2 | ...| ;yes
$x_2$  | 0 | ... | 0 | ... | 0 | ... | 0 | ... | 1 | ... | 1 | ... | 1 | ... | ;no

Step 2. The rest of the NB algorithm works as in the slides, from this point onwards. Let's assume we have a new entry: $x_3:$ "hello world". We vectorize $x_3$, then the algorithm can compute $p(x_3 | spam = "yes")$ and then use Bayes' Theorem to compute $p(y_3="spam" | x_3) $.


In [8]:
# Vectorize words
# First, we are going to create a dictionary of all the words
# contained in the X_train dataset using fit()
vectorizer = CountVectorizer().fit(X_train)
# Now we run the frequency counter using transform(), that counts how many times
# each word appears in each message.
# Note: If X_test contains some words that are not in X_train,
# then those words will be skipped in the transformation step.
# Conceptually, we are saying that we cannot use those new words
# to make predictions, since they didn't appear in our training set
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(X_train_vec.shape)
print(X_test_vec.shape)

(3859, 7004)
(1655, 7004)


In [9]:
# Let's try to visualize the words the vectorizer caught
vectorizer.get_feature_names_out()

array(['00', '000', '008704050406', ..., 'zoom', 'ú1', '〨ud'],
      dtype=object)

In [10]:
# Let's also try to see the numbers it assigned to each word...
print(X_train_vec.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [11]:
# Since the matrix is too big, we can't see if there are any non-zero's in there.
# One way to check is to extract all the unique numbers that are contained
# in the matrix, using function numpy.unique()
np.unique(X_train_vec.toarray())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 15, 18])

## Drumroll.... Time to run the Naive Bayes Classifier

In [12]:
# Let's call the model 'NB' for short
NB = MultinomialNB()
NB.fit(X_train_vec, y_train)

MultinomialNB()

In [13]:
# How many entries were ham vs spam?
NB.class_count_

array([3368.,  491.])

In [14]:
# Accuracy of NB model
y_pred = NB.predict(X_test_vec)
accuracy_score(y_test, y_pred)

0.9885196374622357

In [15]:
# Confusion Matrix
confusion_matrix(y_test,y_pred)

array([[1423,    8],
       [  11,  213]])

## Now, let's use our spam filter to classify some entirely new, never beforeseen text...

In [16]:
emails=[
        "Hi all. How are you liking BA305 so far?",
        "Do you think this spam filter is working well?",
        "congratulations, you became today's lucky winner",
        "WINNER WINNER CHICKEN DINNER"
    ]

In [17]:
# Vectorize the emails
emails_vec = vectorizer.transform(emails)
# Call the predict function, 1 means spam, 0 means not spam
NB.predict(emails_vec)

array([0, 0, 1, 1])

Seems like pretty good accuracy... let's try to trick the spam filter. It seems the keyword "winner" comes up often in spam. How good is our spam filter?

In [18]:
emails2=[
        "congratulations, you became today's lucky winner",
        "Zendaya was the winner of our class project today",
        "Zendaya, I wanted to congratulate you as our winner today",
        "Zendaya, I wanted to congratulate you for winning the class project today"
    ]

In [19]:
# Vectorize the emails
emails2_vec = vectorizer.transform(emails2)
# Call the predict function
NB.predict(emails2_vec)

array([1, 0, 0, 0])

Pretty good! We were able to trick it on line 3, but to be fair, it's not 100% clear to us either, if line 3 is spam or not.

## Bonus: How about we try to use our K-NN classifier from last time, to compare?

In [20]:
# Calling our KNN classifier from last time...
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_vec, y_train)
y_pred = knn.predict(X_test_vec)

# Metrics...
print('Accuracy:', accuracy_score(y_test, y_pred))
confusion_matrix(y_test,y_pred)

Accuracy: 0.916012084592145


array([[1431,    0],
       [ 139,   85]])

In [22]:
knn.predict(emails_vec)

array([0, 0, 0, 0])

In [21]:
knn.predict(emails2_vec)

array([0, 0, 0, 0])

k-NN seems to be underperforming compared to NB, at this classsification task...